# CNN + LSTM
##### I really hope I can wirte this as an academic paper, but also really have no time, so I just list all the key engineering steps below.

### 1.1  Business Scenario


   As a fintech company, one of the most important thing we need to be very careful and sentitive is credit risks evaluation.  Imagine when a coffee shop onwer come to borrow money, though his personal record is pretty, but do we need to also consider coffee industry holistic repayment performance in this customer's geographic area?  The answer is yes, cause marco effect do exist and can be model by this model I build up. 

### 1.2  Problem & Solution.

Thus, I defined our problem as a temporal-spatial time sereis forecasting model. Traditional time series forecasting model, like ARIMA, MA, Facebook Prophet, Linear regression, Boosting Tree Regression, only take temporal feature into account, but igonre geographic attribute behind and thus missed a very valueable peice of information.

To solve this limitation, I borrow idea from several state-of-the-art academic papers published from the coorperation between Uber / DIDI(Chinese Uber) and top tier AI lab in academic feild. 

In general, assume readers of this arcticle have basic machine learning and deep learning background, it is not surprisingly I treat geo-location with its geo-neighbours at a time as an image and use CNN to extract high level spatial features from input data, and then use which as LSTM input corresponding to the time step it belongs to, for making prediction.

As I forementioned, my model will forecasting overall repayment performance according to the given fishnet_id, industry and time t. Before we dive into model, I will explain how I prepare our data to help you understand fishnet, industry and every input.

<img src="files/Model.png">

### 1.3  Input : Data Preparation, preprocessing 

#### 1.3.1 Geographic Index

As we can see the picture above, I first use ARCGIS software to do geographic index. 

Specifically, I first use 'Fishnet' toolkit to make Autralia consists of around 2 millons of 2km by 2km grid which are stored their index as fishnet id and 4 pair of coordinators to represent this squre. 

A tips here is, the size of single grid can be determined by your own situation, in this project, by setting it to 2km, I can get significant inprovement on performance, thus, I have not use this as hyper parameter. But in the future, I will tune this hyper parameter as well.

#### 1.3.2 Geographic neighbours

To model the spatial information, I also use the historical information of target grid's neighbours as input. Here, you can imagine each input to Convoluton Neural Nets(CNN) is a 7 by 7 image, with 48 geo neighbours of target grid which located in the center.

In one words, CNN will extracts high level features and models the spatial correlations among these grids.

#### 1.3.3 Dataset construction

In traditional time series forecasting, we only use targets' historical data as input to make prediction. For example, if you want to predict the sellings of next month, what you input is past few months' selling data. 

Here however, I use three different kind of information to fill into above 7 by 7 image and then stack them together (like image channels) to build up our input dataset.

Given a target grid with its 48 neighbours, the first channel is repayment performance(also our predicting target). For a give month and insudtry, I calculate the overall repayment performance within these 49 fishnet grids and use this 7 by 7 array as one channel of our input.  Then I follow the above way and fill other two 7 by 7 channgel with density and overall dishounours. 

Here, my aim is not introduce our model, but try to inspire reader to do their own project, so I will not discuss many details of what our own data. Just keep in mind that, you can define whatever data you want to suit your scenario.

So, finally, I use 7*7*3 np ndarray to represent this 'image' data at one time, while our target is centre grid's repayment performance at predicting month. 

### 1.4  Models

Models is not very complicate, each input data, as described in above, will be outputted as a falttened features by CNN, which will then as input to LSTM model, like traiditional time series forecasting.

####  1.4.1 Model
I attch my code below as reference,

In [3]:
def cnn_on_lstm_rms_model(CONV2D_1_DEPTH, CONV2D_2_DEPTH, CNN_DP, #CNN archi related
                          LSTM_DP, LSTM_UNITS, #LSTM archi related# RMSprop related
                          inputshapes, loss_fnc):
    model = Sequential()
    #add two time-distributed convolutional layers for feature extraction
    model.add(TimeDistributed(Conv2D(CONV2D_1_DEPTH, (3, 3), activation='relu'), input_shape = inputshapes))
    #model.add(TimeDistributed(Conv2D(16, (4, 4), activation='relu')))
    model.add(TimeDistributed(Conv2D(CONV2D_2_DEPTH, (5, 5), activation='relu')))


    # extract features and dropout 
    model.add(TimeDistributed(Flatten()))
    model.add(Dropout(CNN_DP))
    #print(123123123)
    # input to LSTM
    model.add(LSTM(LSTM_UNITS, return_sequences=True, dropout=LSTM_DP)) #input_shape=(5, 512), 
    #print(123123123)
    # classifier with sigmoid activation for multilabel
    model.add(Dense(1, activation='tanh'))
    #model.add(TimeDistributed(Dense(1, activation='tanh')))

    # compile the model with binary_crossentropy loss for multilabel
    #Rmsprop = keras.optimizers.RMSprop(lr=RMSprop_LR, rho=RMSprop_RHO, 
    #                                   epsilon=None, decay=RMSprop_DECAY)
    #model.compile(optimizer= Rmsprop, loss = loss_fnc)
    #keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    model.compile(optimizer= 'rmsprop', loss = mape_loss)
    # look at the params before training
    #model.summary()
    return model

#### 1.4.2 Hyper Parameters List

In general, there are many hyperparameter to tune if you have enough computing resources and time. Here, I will breifly explain what is our hyperparaments and how I choose specific value for them in the list below,

| Hyperparameter | Chosen Value | Reason | 
| --- | --- | --- |
| CNN 1st filter size  | 3 | our fishnet grid is 2km * 2km, empirically, we make linear combination of features within range of 6km * 6km will enough|
| CNN 2nd filter size  | 5 | similar to above one, but we can consider more wider range based on extracted high level features |
| CNN 1st DEPTH | 64 | Gird search from [32,64,128] |
| CNN 2nd DEPTH | 64 | Gird search from [32,64,128] |
| CNN 1st Active Funtion | relu | avoid gradient vanish |
| CNN 2nd Active Funtion | relu | avoid gradient vanish |
| CNN Drop Out | 0.1 | Gird search from [0.1,0.3,0.5] |
| LSTM Unit | 64 | Gird search from [32,64,128] |
| LSTM Drop Out | 0.1 | Gird search from [0.1,0.3,0.5] |
| LSTM Active Function | tanh | just the default setting of LSTM, especially LSTM not very deep  |

Moreover, in terms of optimzed function, we choose the RMSprop to salute to G.Hinton :) Specifically, it is because our data is sparse and also means it is kind of 'stable', for the sake of speeding up the learning process, we choose algorithms which can employ adptive learning rate based on the updated gradient in each round. Obviously, Adagrad, Adadelta, RMSprop, Adam are common choices. RMSProp and Adadelta are almost the same, excpet Adadelta use RMS of parameters, rather than RMS of initial learning rate, to update weights. There is not obvious different when compare Adam and RMSprop ,thus, we finally choose RMSprop.

### 1.5  Comparison among different models

I did a grid search on all hyper parameters of my CNN+LSTM model, and choose the best one to compare with other three models, which are xgboost regression, Moving average and ARIMA. The comparison result can be seen as below(I only choose 3 main industries),

<img src="files/compare.png" alt="Drawing" style="width: 350px;"/>

So, as we can see MAPE and RMSE compare, my model outperform other three.

### 1.6  Comparison inside differet channel

#### 1.6.1 comparison of MAPE and RMSE

I also did an additional comparison among utilizing 1 channgel only, utilizing 2 channels and utilizing 3 channels. This is also new idea to all reference paper I surveyed and expriment prove that using 3 channel is the best to our scenario. The result can be seen as below,

|Industry_ID | Channel(s) | MAPE | RMSE |
| --- | --- | --- |
| 1 | 1 | 2.99 |0.023 |
| 1 | 2 | 2.71 |0.021 |
| 1 | 3 | 2.67 |0.019 |

|Industry_ID | Channel(s) | MAPE | RMSE |
| --- | --- | --- |
| 2 | 1 | 2.57 |0.02 |
| 2 | 2 | 2.31 |0.018 |
| 2 | 3 | 2.22 |0.017 |

|Industry_ID | Channel(s) | MAPE | RMSE |
| --- | --- | --- |
| 3 | 1 | 3.74 |0.026 |
| 3 | 2 | 3.40 |0.024 |
| 3 | 3 | 3.30 |0.022 |

In all of these three main industries, model with 3 channel input data can beats others - I use the same parameter and hyper parameters for all these models

#### 1.6.2 comparison of error disstribution

Cause majority of our target value is stable, the MAPE and RMSE cannot fully demonstrate the capbility of capturing the performance when predicting some sample with relatively smaller target value.

Thus, I defined a metric named '085 erros distribution', which means choose all the sample with true repayment performance under 0.85 and then count how many times the absolute value of predicting value minus true value less than or equal to 0.1.

As Deep learning models have ramdom parameters which lead to different results, so we run each of these 3 different models 30 times and plot the distributuon of '085 erros distribution'

The comparasion can be seen as below,

In [1]:
from IPython.display import Image, HTML, display
from glob import glob
#imagesList=''.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
#                     for s in sorted(glob('map_*.png')) ])
display(HTML("<table><tr><td><img src='hos1.png'></td><td><img src='hos2.png'></td><td><img src='hos3.png'></td></tr></table><figcaption>Industry_1: 1, 2 and 3 channel</figcaption>"))
display(HTML("<table><tr><td><img src='ret1.png'></td><td><img src='ret2.png'></td><td><img src='ret3.png'></td></tr></table><figcaption>Industry_2: 1, 2 and 3 channel</figcaption>"))
display(HTML("<table><tr><td><img src='bud1.png'></td><td><img src='bud2.png'></td><td><img src='bud3.png'></td></tr></table><figcaption>Industry_3: 1, 2 and 3 channel</figcaption>"))

,,


,,


,,


Apparently, performance of 3 channel in this metric still the best. 

----------- TBC